In [2]:
!pip install pandas-gbq --quiet

In [1]:
import pandas as pd
from pandas_gbq import read_gbq
import os

In [ ]:
query = """
  SELECT *
  FROM bigquery-public-data.chicago_crime.crime
  LIMIT 1000
"""

In [ ]:
df = read_gbq(query, project_id='curso-ebac-487022', dialect='standard')

Downloading: 100%|██████████|


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_key            1000 non-null   Int64              
 1   case_number           1000 non-null   object             
 2   date                  1000 non-null   datetime64[us, UTC]
 3   block                 1000 non-null   object             
 4   iucr                  1000 non-null   object             
 5   primary_type          1000 non-null   object             
 6   description           1000 non-null   object             
 7   location_description  997 non-null    object             
 8   arrest                1000 non-null   boolean            
 9   domestic              1000 non-null   boolean            
 10  beat                  1000 non-null   Int64              
 11  district              1000 non-null   Int64              
 12  ward   

In [ ]:
for col in df.columns:
  print(f"{col}: {df[col].dtype}")

unique_key: Int64
case_number: object
date: datetime64[us, UTC]
block: object
iucr: object
primary_type: object
description: object
location_description: object
arrest: boolean
domestic: boolean
beat: Int64
district: Int64
ward: Int64
community_area: Int64
fbi_code: object
x_coordinate: float64
y_coordinate: float64
year: Int64
updated_on: datetime64[us, UTC]
latitude: float64
longitude: float64
location: object


In [ ]:
df.to_parquet('/content/drive/MyDrive/Colab Notebooks/chicago-crimes-analysis/data/raw/chicago_crime_raw.parquet', index=False)

In [ ]:
raw_path = '/content/drive/MyDrive/Colab Notebooks/chicago-crimes-analysis/data/raw/chicago_crime_raw.parquet'
df = pd.read_parquet(raw_path)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_key            1000 non-null   Int64              
 1   case_number           1000 non-null   object             
 2   date                  1000 non-null   datetime64[us, UTC]
 3   block                 1000 non-null   object             
 4   iucr                  1000 non-null   object             
 5   primary_type          1000 non-null   object             
 6   description           1000 non-null   object             
 7   location_description  997 non-null    object             
 8   arrest                1000 non-null   boolean            
 9   domestic              1000 non-null   boolean            
 10  beat                  1000 non-null   Int64              
 11  district              1000 non-null   Int64              
 12  ward   

In [ ]:
import numpy as np
#limpeza
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(' ', '_', regex=False)
)

In [ ]:
if 'unique_key' in df.columns:
  df = df.drop_duplicates(subset='unique_key')
else:
  df= df.drop_duplicates()

In [ ]:
if 'date' in df.columns:
  df['date'] = pd.to_datetime(df['date'], errors='coerce', utc=False)

In [ ]:
for c in ['arrest', 'domestic']:
  if c in df.columns:
    df[c] = df[c].astype('boolean')

In [ ]:
text_cols = ['block', 'primary_type', 'description', 'location_description']
for c in text_cols:
  df[c] = df[c].astype('string').str.strip()

In [ ]:
codigos_cols = ['case_number', 'iucr', 'fbi_code']
for c in codigos_cols:
  df[c] = df[c].astype('string').str.strip()

In [ ]:
for c in ['latitude', 'longitude']:
  if c in df.columns:
    df[c] = pd.to_numeric(df[c], errors='coerce')

In [ ]:
df['location_description'] = df['location_description'].fillna('unknown')

In [ ]:
df = df.drop(columns=['location'], errors ='ignore')

In [ ]:
df.isnull().sum()

,0
unique_key,0
case_number,0
date,0
block,0
iucr,0
primary_type,0
description,0
location_description,0
arrest,0
domestic,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_key            1000 non-null   Int64              
 1   case_number           1000 non-null   string             
 2   date                  1000 non-null   datetime64[us, UTC]
 3   block                 1000 non-null   string             
 4   iucr                  1000 non-null   string             
 5   primary_type          1000 non-null   string             
 6   description           1000 non-null   string             
 7   location_description  1000 non-null   string             
 8   arrest                1000 non-null   boolean            
 9   domestic              1000 non-null   boolean            
 10  beat                  1000 non-null   Int64              
 11  district              1000 non-null   Int64              
 12  ward   

In [ ]:
df.to_parquet('/content/drive/MyDrive/Colab Notebooks/chicago-crimes-analysis/data/processed/chicago_crime_processed.parquet', index=False)

In [ ]:
processed_path = '/content/drive/MyDrive/Colab Notebooks/chicago-crimes-analysis/data/processed/chicago_crime_processed.parquet'
df_processed = pd.read_parquet(processed_path)



In [ ]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   unique_key            1000 non-null   Int64              
 1   case_number           1000 non-null   string             
 2   date                  1000 non-null   datetime64[us, UTC]
 3   block                 1000 non-null   string             
 4   iucr                  1000 non-null   string             
 5   primary_type          1000 non-null   string             
 6   description           1000 non-null   string             
 7   location_description  1000 non-null   string             
 8   arrest                1000 non-null   boolean            
 9   domestic              1000 non-null   boolean            
 10  beat                  1000 non-null   Int64              
 11  district              1000 non-null   Int64              
 12  ward   

In [ ]:
df_processed['month'] = df_processed['date'].dt.to_period('M')
df_processed['day_of_week'] = df_processed['date'].dt.day_name()
df_processed['hour'] = df_processed['date'].dt.hour

violent_types = [
    'HOMICIDE',
    'ROBBERY',
    'BATTERY',
    'CRIMINAL SEXUAL ASSAULT',
    'ASSAULT'
]

df_processed['is_violent'] = df_processed['primary_type'].isin(violent_types)

/tmp/ipython-input-1228695550.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_processed['month'] = df_processed['date'].dt.to_period('M')


In [ ]:
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df_processed['day_of_week'] = pd.Categorical(df_processed['day_of_week'], categories=order, ordered=True)

In [ ]:
df_processed.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,month,day_of_week,hour,is_violent
0,11308464,JB254831,2018-04-28 20:30:00+00:00,002XX N STATE ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,1176327.0,1901797.0,2018,2018-05-10 15:46:13+00:00,41.885888,-87.627942,2018-04,Saturday,20,False
1,13965403,JJ412670,2025-09-13 00:00:00+00:00,002XX N STATE ST,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,STREET,False,False,...,1176310.0,1902226.0,2025,2025-11-02 15:41:09+00:00,41.887066,-87.627992,2025-09,Saturday,0,True
2,8124180,HT359035,2011-06-21 13:00:00+00:00,005XX W MADISON ST,0312,ROBBERY,ARMED:KNIFE/CUTTING INSTRUMENT,SIDEWALK,False,False,...,1173039.0,1900311.0,2011,2018-02-10 15:50:01+00:00,41.881884,-87.640060,2011-06,Tuesday,13,True
3,13790909,JJ200184,2025-03-30 15:20:00+00:00,0000X E LAKE ST,0313,ROBBERY,ARMED - OTHER DANGEROUS WEAPON,CONVENIENCE STORE,False,False,...,1176905.0,1901744.0,2025,2025-05-17 15:40:52+00:00,41.885730,-87.625821,2025-03,Sunday,15,True
4,11943977,JD106803,2020-01-06 20:38:00+00:00,0000X W RANDOLPH ST,0313,ROBBERY,ARMED: OTHER DANGEROUS WEAPON,HOTEL/MOTEL,False,False,...,1176068.0,1901284.0,2020,2020-01-13 15:41:18+00:00,41.884486,-87.628909,2020-01,Monday,20,True


In [ ]:
df_processed.to_parquet('/content/drive/MyDrive/Colab Notebooks/chicago-crimes-analysis/data/prepared/crimes_chicago_prepared.parquet')

In [ ]:
prepared_path = '/content/drive/MyDrive/Colab Notebooks/chicago-crimes-analysis/data/prepared/crimes_chicago_prepared.parquet'
df_prepared = pd.read_parquet(prepared_path)
df_prepared.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,month,day_of_week,hour,is_violent
0,11308464,JB254831,2018-04-28 20:30:00+00:00,002XX N STATE ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,1176327.0,1901797.0,2018,2018-05-10 15:46:13+00:00,41.885888,-87.627942,2018-04,Saturday,20,False
1,13965403,JJ412670,2025-09-13 00:00:00+00:00,002XX N STATE ST,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,STREET,False,False,...,1176310.0,1902226.0,2025,2025-11-02 15:41:09+00:00,41.887066,-87.627992,2025-09,Saturday,0,True
2,8124180,HT359035,2011-06-21 13:00:00+00:00,005XX W MADISON ST,0312,ROBBERY,ARMED:KNIFE/CUTTING INSTRUMENT,SIDEWALK,False,False,...,1173039.0,1900311.0,2011,2018-02-10 15:50:01+00:00,41.881884,-87.640060,2011-06,Tuesday,13,True
3,13790909,JJ200184,2025-03-30 15:20:00+00:00,0000X E LAKE ST,0313,ROBBERY,ARMED - OTHER DANGEROUS WEAPON,CONVENIENCE STORE,False,False,...,1176905.0,1901744.0,2025,2025-05-17 15:40:52+00:00,41.885730,-87.625821,2025-03,Sunday,15,True
4,11943977,JD106803,2020-01-06 20:38:00+00:00,0000X W RANDOLPH ST,0313,ROBBERY,ARMED: OTHER DANGEROUS WEAPON,HOTEL/MOTEL,False,False,...,1176068.0,1901284.0,2020,2020-01-13 15:41:18+00:00,41.884486,-87.628909,2020-01,Monday,20,True
